In [1]:
import scanpy as sc
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter, defaultdict

In [2]:
FILE = "datasets/10XGenomics/Targeted_SC3v3_Human_Glioblastoma_Neuroscience_filtered_feature_bc_matrix.h5"

In [4]:
adata = sc.read_10x_h5(FILE)
X = adata.X.todense()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
B = lambda i: len(bin(i)) - 2
def cell(i):
    return np.array(X[i, :]).flatten()

In [6]:
ncells = X.shape[0]
cells = []
for i in range(ncells):
    cells.append(cell(i))

In [7]:
c = Counter(np.array(cells).flatten())
del c[0]

In [42]:
N = np.array(X).flatten().shape[0]
O = c[1]

i = 1
body = []
while i in c:
    body.append(c[i])
    i += 1
K = i - 1

T = 0
T1 = []
T2 = []
M = int(max(c))
for j in range(i, M):
    if j not in c:
        continue
    T += 1
    T1.append(j)
    T2.append(c[j])

S = max(T2)

In [19]:
Xf = np.array(cells).flatten()
d = defaultdict(list)
for a, b in enumerate(Xf):
    d[b].append(a)

In [65]:
e = {}
for k, v in d.items():
    if v:
        e[k] = [v[0]] + [v[i + 1] - v[i] for i in range(len(v) - 1)]

In [107]:
R = []
for i in range(101, M + 1):
    if i in d:
        for j in d[i]:
            R.append(j)

In [108]:
R

[45530,
 58130,
 72787,
 76314,
 117095,
 117179,
 133806,
 151553,
 173615,
 177742,
 192165,
 198435,
 202061,
 211490,
 303649,
 327415,
 341005,
 388204,
 418454,
 447155,
 575669,
 603290,
 646449,
 680552,
 717461,
 724577,
 840639,
 871779,
 877495,
 899908,
 905476,
 922639,
 923277,
 946044,
 1002331,
 1003389,
 1031608,
 1084806,
 1189931,
 1233075,
 1274333,
 1304455,
 1324981,
 1329437,
 1332431,
 1348270,
 1383140,
 1392109,
 1440238,
 1454838,
 1486889,
 1515833,
 1522641,
 1702849,
 1704315,
 1839417,
 1870401,
 1883133,
 1971116,
 1971165,
 1981061,
 1998979,
 2049739,
 2119755,
 2130466,
 2257331,
 2273896,
 2317513,
 2356853,
 2358227,
 2358550,
 2402471,
 2411511,
 2465207,
 2515879,
 2522655,
 2575780,
 2584373,
 2606015,
 2641621,
 2650334,
 2692142,
 2746760,
 2763449,
 2775036,
 2779887,
 2781869,
 2832541,
 2868789,
 2941849,
 2954825,
 2958675,
 2962141,
 2994729,
 3070178,
 3093641,
 3131073,
 3154008,
 3157999,
 3164689,
 3173119,
 3216885,
 3225144,
 3233307

In [56]:
38215 * 23

878945

In [55]:
3378317 / 23

146883.34782608695

In [36]:
R[:10]

[0, 8, 24, 37, 38, 79, 121, 172, 180, 201]

In [37]:
bits = len(bin(max(R))) - 2
bits

23

In [38]:
for i in R[:10]:
    print(bin(i)[2:].zfill(bits))

00000000000000000000000
00000000000000000001000
00000000000000000011000
00000000000000000100101
00000000000000000100110
00000000000000001001111
00000000000000001111001
00000000000000010101100
00000000000000010110100
00000000000000011001001


In [111]:
def write_bytes(handle, bits, bytes=4):
    handle.write(int(bits[::-1], 2).to_bytes(bytes, 'little'))

def write_int(handle, value, bytes=4):
    handle.write(value.to_bytes(bytes, 'little'))

def write_array(handle, array):
    bits = len(bin(max(array))) - 2
    write_int(handle, len(array))
    write_int(handle, bits)

    # print(len(array), bits, (len(array) * 23) - (len(array) * bits + 2 * 32 * 8))
    curr_str = ""
    for elem in array:
        s = bin(elem)[2:].zfill(bits)
        for char in s:
            curr_str += char
            if len(curr_str) == 32:
                write_bytes(handle, curr_str)
                curr_str = ""
    if curr_str:
        write_bytes(handle, curr_str)

    return len(array) * bits + 2 * 32 * 8

In [112]:
with open("10x.conq", "wb") as k:
    total = 0
    for i in range(1, 100):
        total += write_array(k, e[i])
    # print(total)
    # print(write_array(k, R))
    total += write_array(k, R)
    total += write_array(k, body)
    total += write_array(k, T1)
    total += write_array(k, T2)
    print(total)

19948634
